In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from my_functions import *

Esto es un resumen de los 5 archivos con el fin de presentar información directa

----
## Preparamos todas las tablas y datos que ocuparemos

In [2]:
# Guardamos de la API las tablas necesarias (Archivo 01)
usd_b = get_simple_table('usd')                 # Valor del dolar blue en soles
usd_o = get_simple_table('usd_of')              # Valor del dolar oficial en soles
events = get_simple_table('milestones')         # Eventos importantes
usd_b.to_csv('API_tables/usd_b.csv', index = False)
usd_o.to_csv('API_tables/usd_o.csv', index = False)
events.to_csv('API_tables/events.csv', index = False)

# Creamos tabla principal (Archivo 03)
dolar_b = pd.read_csv('API_tables/usd_b.csv')
dolar_o = pd.read_csv('API_tables/usd_o.csv')
eventos = pd.read_csv('API_tables/events.csv')
dolar_b['d'] = pd.to_datetime(dolar_b['d'])
dolar_o['d'] = pd.to_datetime(dolar_o['d'])
eventos['d'] = pd.to_datetime(eventos['d'])
primer_dia = '2002-03-04'
ayer = dt.datetime.strftime(dt.datetime.today() - dt.timedelta(days=1), '%Y-%m-%d')
table = pd.DataFrame(pd.date_range(start=primer_dia, end=ayer), columns=['fecha'])
table['dia_semana'] = table['fecha'].map(lambda x: dt.datetime.strftime(x,'%A'))
table['semana'] = table['fecha'].map(lambda x: dt.datetime.strftime(x,'%Y-%W'))
table = table.join(dolar_b.set_index('d'), on='fecha', how='left').rename(columns={'v': 'dolar_b'})
table = table.join(dolar_o.set_index('d'), on='fecha', how='left').rename(columns={'v': 'dolar_o'})
table['dolar_b'] = table['dolar_b'].interpolate()
table['dolar_o'] = table['dolar_o'].interpolate()
table['brecha_s'] = table['dolar_b'] - table['dolar_o']
table['brecha'] = (table['dolar_b']/table['dolar_o'] - 1)*100
table = add_variation(table, ['dolar_b', 'dolar_o'])
table.rename(columns={'dolar_b_var': 'db_var','dolar_o_var': 'do_var'}, inplace = True)
br_var = np.array([])
for i in table.index:
    if i == 0:
        var = 0
    else:
        var = table.loc[i,'brecha'] - table.loc[i-1,'brecha']
    br_var = np.append(br_var, var)
table['brecha_var'] = br_var
table = table.join(eventos.set_index('d'), on='fecha', how='left').rename(columns={'e': 'evento_p', 't':'evento_t'})
table['evento_p'] = table['evento_p'].fillna('') 
table['evento_t'] = table['evento_t'].fillna('')
table.to_csv('my_tables/table_hist.csv', index= False) # Guardamos tabla principal

# (Archivo 04)
table = pd.read_csv('my_tables/table_hist.csv')
table['fecha'] = pd.to_datetime(table['fecha'])
table['evento_p'] = table['evento_p'].fillna('') 
table['evento_t'] = table['evento_t'].fillna('')
primer_dia = dt.datetime.today() - dt.timedelta(days=366)
table_365 = table[table['fecha'] > primer_dia].copy()
table_365.reset_index(inplace= True, drop=True)
table_365['brecha_var_abs'] = table_365['brecha_var'].abs()
table_365['db_var_abs'] = table_365['db_var'].abs()
table_365['do_var_abs'] = table_365['do_var'].abs()
top_1_brech_var = table_365.sort_values(['brecha_var_abs'], ascending=False)[['fecha', 'brecha_var', 'brecha_var_abs', 'evento_t']].head(1)
top_1_brech_var.reset_index(drop=True, inplace = True)
top_5_var_db = table_365.sort_values(['db_var_abs'], ascending=False)[['fecha', 'dolar_b', 'db_var', 'db_var_abs', 'evento_t', 'evento_p']].head(5)
top_5_var_do = table_365.sort_values(['do_var_abs'], ascending=False)[['fecha', 'dolar_o', 'do_var', 'do_var_abs', 'evento_t', 'evento_p']].head(5)
top_week_brecha_var = table_365[['semana', 'brecha_var_abs', 'brecha']].groupby('semana').mean()
top_week_brecha_var.reset_index(inplace = True)
top_week_brecha_var = top_week_brecha_var.sort_values('brecha_var_abs', ascending=False).head(1)
top_week_brecha_var.reset_index(inplace = True, drop=True)
days = table_365[table_365['semana'] == top_week_brecha_var.loc[0,'semana']][['fecha', 'semana', 'brecha', 'brecha_var', 'brecha_var_abs','evento_p', 'evento_t']]
days.reset_index(drop = True, inplace = True)
days_of_week = table_365[['dia_semana', 'brecha', 'brecha_var']].groupby('dia_semana').mean()
days_of_week.sort_values('brecha_var', inplace = True, ascending=False)
days_of_week.reset_index(inplace = True)
days_of_week_abs = table_365[['dia_semana', 'brecha', 'brecha_var_abs']].groupby('dia_semana').mean()
days_of_week_abs.sort_values('brecha_var_abs', inplace = True, ascending=False)
days_of_week_abs.reset_index(inplace = True)

# (Archivo 05)
eventos = table[table['evento_t'] != '']

day_0_b = dt.datetime.strptime('2016-05-01', '%Y-%m-%d')
table_reg_1 = table.copy()
table_reg_1['fecha_o'] = table_reg_1['fecha'].map(lambda x : date_to_x(x, day_0_b))
table_reg_1.dropna(inplace = True)                                                
table_reg_1 = table_reg_1[['fecha', 'fecha_o', 'dolar_b']]
table_reg_1.head()
X = table_reg_1['fecha_o'].values.reshape(-1,1)
y = table_reg_1['dolar_b'].values
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, y, test_size=0.30, random_state = 0)
dolar_b_lr1 = LinearRegression(fit_intercept=True)
dolar_b_lr1.fit(X_train_b, y_train_b)
y_predict_train_b = dolar_b_lr1.predict(X_train_b)
y_predict_b = dolar_b_lr1.predict(X_test_b)
fechas_b = [x_to_date(x, day_0_b) for x in X_test_b[:,0]]

day_0_o = dt.datetime.strptime('2013-10-01', '%Y-%m-%d')
table_reg_3 = table.copy()
table_reg_3['fecha_o'] = table_reg_3['fecha'].map(lambda x : date_to_x(x, day_0_o))
table_reg_3.dropna(inplace = True)
table_reg_3 = table_reg_3[['fecha', 'fecha_o', 'dolar_o']]
X = table_reg_3['fecha_o'].values.reshape(-1,1)
y = table_reg_3['dolar_o'].values
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X, y, test_size=0.30, random_state = 0)
dolar_o_lr1 = LinearRegression(fit_intercept=True)
dolar_o_lr1.fit(X_train_o, y_train_o)
y_predict_train_o = dolar_o_lr1.predict(X_train_o)
y_predict_o = dolar_o_lr1.predict(X_test_o)
fechas_o = [x_to_date(x, day_0_o) for x in X_test_o[:,0]]

today = dt.datetime.today()
fechas_a_predecir = np.array([])
fechas_a_predecir = np.append(fechas_a_predecir, today + dt.timedelta(days=90)) # 3 meses
fechas_a_predecir = np.append(fechas_a_predecir, today + dt.timedelta(days=180)) # 6 meses
fechas_a_predecir = np.append(fechas_a_predecir, today + dt.timedelta(days=360)) # 12 meses
x_b_predict = np.array([date_to_x(x, day_0_b) for x in fechas_a_predecir])
y_b_predict = dolar_b_lr1.predict(x_b_predict.reshape(-1,1))
x_o_predict = np.array([date_to_x(x, day_0_o) for x in fechas_a_predecir])
y_o_predict = dolar_o_lr1.predict(x_o_predict.reshape(-1,1))

first_day = dt.datetime.today()-dt.timedelta(days = 1459) # 4 años
table_4_anios = table[table['fecha'] > first_day][['fecha', 'brecha_s', 'brecha']]

----
----
## 1) Día con mayor variación en la brecha
(Últimos 365 días)

In [3]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var'], mode='lines', name='Variación'))
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var_abs'], mode='lines', name='Variación absoluta',  line=dict(dash='dot')))
fig.add_trace(go.Scatter(x = top_1_brech_var['fecha'], y = top_1_brech_var['brecha_var'], mode='markers', name='Top variación de brecha'))
fig.update_layout(
    title='Variación de la brecha',
    xaxis_title = 'Fecha',
    yaxis_title = 'Variación del porcentaje')

In [4]:
top_1_brech_var

,fecha,brecha_var,brecha_var_abs,evento_t
0,2022-07-21,15.015052,15.015052,


El día con **mayor variación en la brecha** (tanto normal como absoluta) fue el día **21 de Julio del 2021** con un cambio en la brecha porcentual de **15.01 puntos**

----
----
## 2) Top 5 días con mayor variación
(Últimos 365 días)

----
### ......Para el Dólar Blue:

In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['db_var'], mode='lines', name='Variación del dólar'))
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['db_var_abs'], mode='lines', name='Variación absoluta del dólar',  line=dict(dash='dot')))
fig.add_trace(go.Scatter(x = top_5_var_db['fecha'], y = top_5_var_db['db_var_abs'], mode='markers', name='Top variación absoluta del dólar'))
fig.update_layout(
    title='Variación del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['dolar_b'], mode='lines', name='Valor del Dólar Blue'))
fig.add_trace(go.Scatter(x = top_5_var_db['fecha'], y = top_5_var_db['dolar_b'], mode='markers', name='Top 5 variación absoluta del dólar'))
fig.update_layout(
    title='Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

El **top 5 variaciones el dólar blue** se encuentra en la siguiente tabla:

In [7]:
top_5_var_db

,fecha,dolar_b,db_var,db_var_abs,evento_t,evento_p
350,2022-07-21,337.0,6.309148,6.309148,,
337,2022-07-08,273.0,6.225681,6.225681,,
358,2022-07-29,296.0,-5.732484,5.732484,,
349,2022-07-20,317.0,5.315615,5.315615,,
176,2022-01-28,212.5,-4.494382,4.494382,,


Cuatro del Top 5 variaciones absolutas del dólar blue fueron en **Julio del 2022**. Tres de las 5 corresponden a variaciones positivas. No se observan eventos relevantes.

----
### ......Para el Dólar Oficial:

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['do_var'], mode='lines', name='Variación del dólar'))
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['do_var_abs'], mode='lines', name='Variación absoluta del dólar',  line=dict(dash='dot')))
fig.add_trace(go.Scatter(x = top_5_var_do['fecha'], y = top_5_var_do['do_var_abs'], mode='markers', name='Top variación absoluta del dólar'))
fig.update_layout(
    title='Variación del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['dolar_o'], mode='lines', name='Valor del Dólar Oficial'))
fig.add_trace(go.Scatter(x = top_5_var_do['fecha'], y = top_5_var_do['dolar_o'], mode='markers', name='Top 5 variación absoluta del dólar'))
fig.update_layout(
    title='Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

El **top 5 variaciones el dólar oficial** se encuentra en la siguiente tabla:

In [10]:
top_5_var_do

,fecha,dolar_o,do_var,do_var_abs,evento_t,evento_p
356,2022-07-27,130.89,0.206706,0.206706,,
349,2022-07-20,129.39,0.201347,0.201347,,
323,2022-06-24,124.26,0.185439,0.185439,,
357,2022-07-28,131.13,0.183360,0.183360,econ,Sergio Massa
362,2022-08-02,132.18,0.181901,0.181901,,


Tres del top 5 se encuentran en **Julio del 2022**. La variación top 4 coincide con el nombramiento de Sergio Massa como mínistro de Economía

----
----
## 3) Semana con mayor variación en la brecha
(Últimos 365 días)

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha'], mode='lines', name='Brecha'))
fig.add_trace(go.Scatter(x = days['fecha'], y = days['brecha'], mode='lines', name='Top semana de variación en la brecha'))
fig.update_layout(
    title='Brecha',
    xaxis_title = 'Fecha',
    yaxis_title = 'Puntos porcentuales')

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var_abs'], mode='lines', name='Variación absoluta deBrecha'))
fig.add_trace(go.Scatter(x = days['fecha'], y = days['brecha_var_abs'], mode='lines', name='Top semana de variación en la brecha'))
fig.update_layout(
    title='Variación absoluta de la Brecha',
    xaxis_title = 'Fecha',
    yaxis_title = 'Puntos porcentuales')

La semana se compone de los siguientes días:

In [13]:
days

,fecha,semana,brecha,brecha_var,brecha_var_abs,evento_p,evento_t
0,2022-07-18,2022-29,125.756400,-0.909921,0.909921,,
1,2022-07-19,2022-29,133.098428,7.342028,7.342028,,
2,2022-07-20,2022-29,144.995749,11.897321,11.897321,,
3,2022-07-21,2022-29,160.010802,15.015052,15.015052,,
4,2022-07-22,2022-29,160.480888,0.470086,0.470086,,
5,2022-07-23,2022-29,155.949938,-4.530949,4.530949,,
6,2022-07-24,2022-29,151.433839,-4.516100,4.516100,,


La semana con mayor promedio en la variación absoluta de la brecha fue la **semana del 18 de Julio de 2022** con un promedio de **6.38%**. <br>
No hay eventos destacables en la semana <br>

----
----
## 4) Día de la semana donde hay mayor variación en la brecha
(Últimos 365 días)

Variación de la Brecha:

In [14]:
days_of_week

,dia_semana,brecha,brecha_var
0,Wednesday,93.013229,0.896068
1,Tuesday,92.117161,0.877584
2,Thursday,92.873195,-0.015066
3,Monday,91.239577,-0.269942
4,Sunday,91.509519,-0.270495
5,Saturday,91.780013,-0.271050
6,Friday,92.051063,-0.312178


Variación absoluta de la Brecha

In [15]:
days_of_week_abs

,dia_semana,brecha,brecha_var_abs
0,Wednesday,93.013229,1.864806
1,Friday,92.051063,1.811780
2,Tuesday,92.117161,1.579205
3,Thursday,92.873195,1.456639
4,Saturday,91.780013,0.661811
5,Sunday,91.509519,0.660181
6,Monday,91.239577,0.658558


Tanto en la variación como en la variación absoluta de la brecha obtuvimos que **el día de la semana con mayor promedio de variación es el Miércoles**.

----
----
## 5) Con la info histórica del valor del dólar y del blue, realizar un análisis exploratorio.
Cruzar la data con sucesos importantes a nivel político-económico y graficar mes a mes.

In [16]:
px.bar(x = eventos['dia_semana'].value_counts().index, 
       y = eventos['dia_semana'].value_counts().values/len(eventos), 
       labels={'y':'Porcentaje','x':'Día de la semana'}, 
       title='Ocurrencias de eventos')

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['dolar_b'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['dolar_b'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Valor del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['db_var'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['db_var'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Variación del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['dolar_o'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['dolar_o'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Valor del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['do_var'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['do_var'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Variación del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

In [21]:
eventos[eventos['fecha'] == '2015-12-17']

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t
5042,2015-12-17,Thursday,2015-50,14.36,13.76,0.6,4.360465,-1.441318,39.979654,-43.85927,Levantamiento del Cepo al Dolar,misc


In [22]:
eventos[eventos['fecha'] == '2019-07-01']

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t
6335,2019-07-01,Monday,2019-26,43.45,42.14,1.31,3.108685,-0.267789,-0.244615,-0.023959,Hernán Lacunza,trea


**Obsevaciones**
- Los eventos ocurren los **lunes** cerca de un 30% de las veces.
- Un evento de **julio 2019** pareció elevar el **valor de ambos dólares**.
- Un evento del **diciembre 2015** coincide con una la mayor **variación del dólar oficial**.
- El **Levantamiento del Cepo al Dolar** está lugado con la **mayor variación al dolar oficial** en la historia.
- El **nombramiento de Hernán Lacuzna** como ministro de Hacienda pudo ser detonante de aumento en el valor de ambos dólares de Júlio 2019.

----
----
## 6) Implementar una regresión lineal (una para cada tipo de dólar) para predecir el valor del dólar en:
* 3 meses
* 6 meses
* 12 meses

----
### ......Predicción para Dólar Blue

In [23]:
print('Error en datos de train:', mean_squared_error(y_train_b, y_predict_train_b))
print('Error en datos de test:', mean_squared_error(y_test_b, y_predict_b))
fig = go.Figure()
fig.add_trace(go.Scatter(x = fechas_b, y = y_test_b, mode='markers', name = 'Test data'))
fig.add_trace(go.Scatter(x = fechas_b, y = y_predict_b, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

Error en datos de train: 202.98311298283014
Error en datos de test: 239.4057920932903


**¡El error de los datos pasó de 1622.71 a 239.40!** por usar componente cuadrático.<br>
Procedo a hacer las predicciones:

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_reg_1['fecha'], y = table_reg_1['dolar_b'], mode='lines', name = 'Actual_data'))
fig.add_trace(go.Scatter(x = fechas_a_predecir, y = y_b_predict, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [25]:
print("Predicciones del valor del dólar blue:")
for index in range(len(fechas_a_predecir)):
    print('Predicción para {} : {}'.format(dt.datetime.strftime(fechas_a_predecir[index], '%Y-%m-%d'), y_b_predict[index]))

Predicciones del valor del dólar blue:
Predicción para 2022-11-03 : 265.3684780197765
Predicción para 2023-02-01 : 285.5156762422131
Predicción para 2023-07-31 : 328.0560444129565


----
### ......Predicción para Dólar Oficial

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = fechas_o, y = y_test_o, mode='markers', name = 'Test data'))
fig.add_trace(go.Scatter(x = fechas_o, y = y_predict_o, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

**¡El error de los datos pasó de 397.68 a 26.00!** por usar componente cuadrático<br>
Procedo a hacer las predicciones:

In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_reg_3['fecha'], y = table_reg_3['dolar_o'], mode='lines', name = 'Actual_data'))
fig.add_trace(go.Scatter(x = fechas_a_predecir, y = y_o_predict, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [28]:
print("Predicciones del valor del dólar oficial:")
for index in range(len(fechas_a_predecir)):
    print('Predicción para {} : {}'.format(dt.datetime.strftime(fechas_a_predecir[index], '%Y-%m-%d'), y_o_predict[index]))

Predicciones del valor del dólar oficial:
Predicción para 2022-11-03 : 126.23078384751207
Predicción para 2023-02-01 : 133.0905378618852
Predicción para 2023-07-31 : 147.36045705968957


----
----
## 7) Bonus opcional: Realizar una calculadora de predicción de aumento del dólar

In [29]:
def cal_predict_dolar(fecha, tipo = 'blue'):
    fecha = dt.datetime.strptime(fecha, '%Y-%m-%d')
    if tipo == 'blue':
        day_0 = dt.datetime.strptime('2016-05-01', '%Y-%m-%d') 
        x = date_to_x(fecha, day_0)
        y = dolar_b_lr1.predict(np.array([[x]]))
    elif tipo == 'oficial':
        day_0 = dt.datetime.strptime('2013-10-01', '%Y-%m-%d') 
        x = date_to_x(fecha, day_0)
        y = dolar_o_lr1.predict(np.array([[x]]))
    else:
        return None
    return y[0]

In [30]:
print(cal_predict_dolar('2022-11-03', 'oficial'))
print(cal_predict_dolar('2022-11-03', 'blue'))

126.23078384751207
265.3684780197765


----
----
## 8) Mejor momento para comprar dolár oficial y venderlo a dolár blue
(Últimos 4 años)

Top Brecha en Soles:

In [31]:
table_4_anios.sort_values('brecha_s', ascending = False).head(1)

,fecha,brecha_s,brecha
7454,2022-07-22,208.24,160.480888


Top Brecha en Porcentaje:

In [32]:
table_4_anios.sort_values('brecha', ascending = False).head(1)

,fecha,brecha_s,brecha
7454,2022-07-22,208.24,160.480888


**Conclusión**: Resulta coincidir en que el **22 de Julio de 2022** fue el **mejor momento de los últimos 4 años para comprar Oficial y vender Blue** tanto en porcentaje como en soles. <br>
* Mejor en porcentaje: El retorno de inversión es el mayor (160%)
* Mejor en soles: La cantidad de soles ganados por transacción es la máxima ($208.4 soles)